In [15]:
import sys
from pathlib import Path
sys.path.append("../1_code") 
import random

from libraries.imports import *  
from stage4_Val_indiv import extract_pdf_text_by_code_and_section

Extract text containing only code E

In [16]:
pdf_path = "../0_data/pdfs/2.2 BoD Report 2024 EN -14.03.2025_FINAL.pdf"
result_df = extract_pdf_text_by_code_and_section(pdf_path)

[Section Miss] Row 3, Code='E1', Section='nan' — no matches found.
[Section Match] Row 4, Code='E1-1', Section='paragraph 14', Triggered Page=204
[Section Match] Row 4, Code='E1-1', Section='paragraph 14', Triggered Page=205
[Section Miss] Row 4, Code='E1-1', Section='paragraph 16 x x (g)' — no matches found.
[Section Miss] Row 5, Code='E1-2', Section='nan' — no matches found.
[Section Miss] Row 6, Code='E1-4', Section='paragraph 34' — no matches found.
[Section Miss] Row 7, Code='E1-5', Section='paragraph 38' — no matches found.


In [17]:
result_df

,Matched Code,Relevant Pages (Raw),Used Pages,Section Reference,Extracted Text
0,E1-1,105,105,None,105 8.2. Environmental Information Climate Cha...
1,E1-1,105,204,paragraph 14,paragraph 14 x Not material ESRS E3-4 Total wa...
2,E1-1,105,205,paragraph 14,paragraph 14 (f) x 148 ESRS 2- SBM3 - S1 Risk ...
3,E1-2,65,65,None,65 BP 1-5-d BP 2-5- 15 BP 2-5- 17 BP 1-5-c BP-...
4,E1-4,"123,124,125,126",123,None,123 prioritize financing in sustainable sector...
5,E1-4,"123,124,125,126",124,None,124 Achieve a 55% reduction in the carbon foot...
6,E1-4,"123,124,125,126",125,None,125 BRD Group has established 2019 as the base...
7,E1-4,"123,124,125,126",126,None,126 will primarily contribute to the reduction...


In [18]:
#preview
for i, row in result_df.iterrows():
    print(f"\n--- Code: {row['Matched Code']} | Page: {row['Used Pages']} | Section: {row['Section Reference']} ---\n")
    print(row["Extracted Text"][:1000])  # preview first 1000 chars


--- Code: E1-1 | Page: 105 | Section: None ---

105 8.2. Environmental Information Climate Change This chapter includes details on the identified impacts, risks and opportunities (IROs) related to climate change, as well as the corresponding policies, actions and targets. Furthermore, BRD Group’s overview on the policies, measures and targets is also presented. Strategy E1-1-17 Transition Plan BRD Group acknowledges the importance of having a transition plan to align with climate goals and support the sustainability transition. While BRD Group does not currently have a formal transition plan in place, it is committed to developing and adopting one within its report for 2027. This timeline reflects the complexity and scope of the transition process, ensuring that the plan will be comprehensive, actionable and aligned with both regulatory requirements and the broader strategic objectives of SG. BRD Group is closely aligning its sustainability initiatives with the overarching strategy of

In [19]:
#save results in folder extracted_text_indiv
text_only = result_df["Extracted Text"]

company_name = Path(pdf_path).stem.strip()
save_path = Path("../2_output/extracted_text_indiv") / f"{company_name}_extracted_text_only.txt"
save_path.parent.mkdir(parents=True, exist_ok=True)

text_only.to_csv(save_path, index=False, header=False)
print(f"Saved extracted text only to: {save_path}")

Saved extracted text only to: ../2_output/extracted_text_indiv/2.2 BoD Report 2024 EN -14.03.2025_FINAL_extracted_text_only.txt


Alternatively, run through all pdfs in excel

In [ ]:
# === Load all report names from Excel ===
REPO_ROOT = Path().resolve()
excel_path = REPO_ROOT.parent / "0_data" / "table_extraction_accuracy2.xlsx"
sheet_name = "suggested weights"
pdf_df = pd.read_excel(excel_path, sheet_name=sheet_name)

# Match full filename with extension
pdf_df = pdf_df.dropna(subset=["pdf_path", "emission level"])
pdf_df["pdf_name"] = pdf_df["pdf_path"].apply(lambda p: Path(p).name)
pdf_names = pdf_df["pdf_name"].unique()
emission_map = dict(zip(pdf_df["pdf_name"], pdf_df["emission level"]))

pdf_folder = Path("../0_data/pdfs")
output_folder = Path("../2_output/extracted_text_indiv")
output_folder.mkdir(parents=True, exist_ok=True)

processed_count = 0
error_count = 0
high_emission_count = 0
low_emission_count = 0

# === Process each matching PDF ===
for pdf_path in pdf_folder.glob("*.pdf"):
    if pdf_path.name in pdf_names:
        try:
            print(f"🔍 Processing: {pdf_path.name}")
            result_df = extract_pdf_text_by_code_and_section(pdf_path)
            text_only = result_df["Extracted Text"]

            save_path = output_folder / f"{pdf_path.stem}_extracted_text_only.txt"
            text_only.to_csv(save_path, index=False, header=False)
            print(f"Saved: {save_path}")
            processed_count += 1

            # Count by emission category
            emission_label = emission_map.get(pdf_path.name, "").strip().lower()
            if emission_label == "high":
                high_emission_count += 1
            elif emission_label == "low":
                low_emission_count += 1

        except Exception as e:
            print(f"❌ Error processing {pdf_path.name}: {e}")
            error_count += 1
    else:
        print(f"⏩ Skipped (not in Excel): {pdf_path.name}")


🔍 Processing: UniCredit.pdf
❌ Error processing UniCredit.pdf: 'Extracted Text'
🔍 Processing: AB InBev 2024 Annual Report FINAL_Interactive.pdf
❌ Error processing AB InBev 2024 Annual Report FINAL_Interactive.pdf: Cannot find the standardized table at: ../2_output/standardized_merged_by_company/AB InBev 2024 Annual Report FINAL_Interactive_standardized_full.csv
🔍 Processing: Covestro_2024_GB_EN.pdf
[Section Miss] Row 3, Code='E2', Section='nan' — no matches found.
[Section Miss] Row 4, Code='E3', Section='nan' — no matches found.
❌ Error processing Covestro_2024_GB_EN.pdf: 'Extracted Text'
🔍 Processing: consolidated-annual-report-endesa-2024.pdf
❌ Error processing consolidated-annual-report-endesa-2024.pdf: 'Extracted Text'
🔍 Processing: Amorim_RC24_EN.pdf
❌ Error processing Amorim_RC24_EN.pdf: 'Extracted Text'
🔍 Processing: Valeo_2024-Universal-registration-document.pdf
[Section Match] Row 1, Code='E1-3', Section='4.2.3.6', Triggered Page=210
[Section Match] Row 1, Code='E1-3', Section

In [ ]:
# === Summary ===
print(f"\nTotal processed: {processed_count} PDF(s)")
print(f"Total errors: {error_count}")
print(f"High-emission reports processed: {high_emission_count}")
print(f"Low-emission reports processed: {low_emission_count}")

The current number of processed pdfs is 42, because not all standardized tables have code E, and some pdf failed in stage 2 to extract the table content at all

Now create separate txt file for high and low emission firms

In [ ]:
REPO_ROOT = Path().resolve()
excel_path = REPO_ROOT.parent / "0_data" / "table_extraction_accuracy2.xlsx"
sheet_name = "suggested weights"
pdf_df = pd.read_excel(excel_path, sheet_name=sheet_name)

pdf_df = pdf_df[["pdf_path", "emission level"]].dropna()
pdf_df["pdf_name"] = pdf_df["pdf_path"].apply(lambda p: Path(p).stem.strip())
emission_map = dict(zip(pdf_df["pdf_name"], pdf_df["emission level"]))

# === Input and output folders ===
input_folder = Path("../2_output/extracted_text_indiv")
output_folder = Path("../2_output/extracted_text_indiv")
output_folder.mkdir(parents=True, exist_ok=True)

# === Prepare containers ===
grouped = {"High": [], "Low": []}

# === Loop through extracted txt files ===
for file in input_folder.glob("*_extracted_text_only.txt"):
    stem = file.stem.replace("_extracted_text_only", "")
    label = emission_map.get(stem)

    if label in grouped:
        content = file.read_text(encoding="utf-8").strip()
        if content:
            grouped[label].append(f"--- {stem} ---\n{content}")
    else:
        print(f"⚠️ Skipped {stem}: No emission label found")

# === Save grouped files ===
for label, texts in grouped.items():
    out_path = output_folder / f"{label}_Emission_total.txt"
    out_path.write_text("\n\n".join(texts), encoding="utf-8")
    print(f"Saved grouped file: {out_path}")